In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd drive/MyDrive/test-paper-detector/detectron2/research

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from PIL import Image

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
!pip install tf_slim

In [ ]:
import os
import tensorflow as tf

PATH_TO_FROZEN_GRAPH = 'inference_graph_choice/frozen_inference_graph.pb'
 
detection_graph = tf.Graph()
with detection_graph.as_default():
 
    od_graph_def = tf.compat.v1.GraphDef()
 
    with tf.compat.v2.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as f:
 
        serialized_graph = f.read()
        od_graph_def.ParseFromString(serialized_graph)
 
        tf.import_graph_def(od_graph_def, name = "")
print('\n계산 그래프 로드 완료...\n')
 
 
 
import cv2
import numpy as np
 
def run_inference_for_single_image(image, graph):
    with tf.compat.v1.Session(graph = graph) as sess:
 
        input_tensor = graph.get_tensor_by_name('image_tensor:0')
        
        target_operation_names = ['num_detections', 'detection_boxes',
                                  'detection_scores', 'detection_classes', 'detection_masks']
        tensor_dict = {}
        for key in target_operation_names:
            op = None
            try:
                op = graph.get_operation_by_name(key)
                
            except:
                continue
 
            tensor = graph.get_tensor_by_name(op.outputs[0].name)
            tensor_dict[key] = tensor
 
        if 'detection_masks' in tensor_dict:
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
 
        output_dict = sess.run(tensor_dict, feed_dict = {input_tensor : [image]})
            
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
 
        return output_dict
   
PATH_TO_TEST_IMAGES_DIR = 'object_detection/test_images'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image_{}.jpg'.format(i)) for i in range(272,273) ] #91
print('분석 대상 이미지 경로 지정 완료...')
 
for image_path in TEST_IMAGE_PATHS:    
    image_np = cv2.imread(image_path)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    height, width, _ = image_np.shape
    
    output_dict = run_inference_for_single_image(image_np, detection_graph)



계산 그래프 로드 완료...

분석 대상 이미지 경로 지정 완료...


In [ ]:
output_dict

{'detection_boxes': array([[0.81998897, 0.06863274, 0.8983147 , 0.11531739],
        [0.6460957 , 0.06991642, 0.7184408 , 0.11508442],
        [0.46760225, 0.48646086, 0.53514314, 0.53071666],
        [0.60241073, 0.4818774 , 0.7174604 , 0.53703713],
        [0.46989894, 0.06889998, 0.53900415, 0.11594595]], dtype=float32),
 'detection_classes': array([1, 1, 1, 2, 1], dtype=uint8),
 'detection_scores': array([0.9880084 , 0.98097265, 0.9792619 , 0.97889143, 0.9697799 ],
       dtype=float32),
 'num_detections': 5}

In [ ]:
import cv2
from operator import itemgetter

def draw_bounding_boxes(img, output_dict, class_info):
    height, width, _ = img.shape
 
    obj_index = output_dict['detection_scores'] > 0.0
    
    scores = output_dict['detection_scores'][obj_index]
    boxes = output_dict['detection_boxes'][obj_index]
    classes = output_dict['detection_classes'][obj_index]
    count = 0

    boxlist = []
    
    for box, cls in zip(boxes, classes):
        boxlist.append([box[0], box[1], box[2], box[3], cls])

    option = 0
    cp = 0
    for i in range(5):
      for j in range(5):
        if(boxlist[i][2] < boxlist[j][0]):
          cp += 1
    print(cp)
    if(cp > 6 and cp < 8): # 두줄
      option = 2
    elif(cp == 8): #세줄
      option = 3
    
    print(boxlist)
    temp = []
    if (option == 0):
      temp = boxlist
      temp.sort(key=lambda x:x[1])

    elif (option == 2): #y축 배열 후 해당 행에서 다시 비교
      print(option)
      boxlist.sort(key=lambda x:x[0])
      for i in range(5):
        if(boxlist[i][4] == 2):
          if(i == 3 or i == 4):
            temp.append(boxlist[0])
            temp.append(boxlist[1])
            temp.append(boxlist[2])
            del boxlist[0]
            del boxlist[0]
            del boxlist[0]
            boxlist.sort(key=lambda x:x[1])
            temp.append(boxlist[0])
            temp.append(boxlist[1])
            break

          elif(i == 0 or i == 1 or i == 2):
            del boxlist[3]
            del boxlist[3]
            boxlist.sort(key=lambda x:x[1])
            temp.append(boxlist[0])
            temp.append(boxlist[1])
            temp.append(boxlist[2])
            break

    else: #y축 배열 후 해당 행에서 다시 비교
      print(option)
      boxlist.sort(key=lambda x:x[0])
      print(boxlist)
      for i in range(5):
        print(i)
        if(boxlist[i][4] == 2):
          print(i)
          if(i == 0 or i == 1):
            del boxlist[2]
            del boxlist[2]
            boxlist.sort(key=lambda x:x[1])
            temp.append(boxlist[0])
            temp.append(boxlist[1])
            temp.append(boxlist[2])
            break

          elif(i == 2 or i == 3):
            temp.append(boxlist[0])
            temp.append(boxlist[1])
            del boxlist[0]
            del boxlist[0]
            del boxlist[2]
            boxlist.sort(key=lambda x:x[1])
            temp.append(boxlist[0])
            temp.append(boxlist[1])
            break

          else:
            temp = boxlist
            break


    print(temp)

    for box in temp:
        count += 1
        if(box[4] == 2):
          print(count,"번을 선택했습니다.")

        img = cv2.rectangle(img,
                            (int(box[1] * width), int(box[0] * height)),
                            (int(box[3] * width), int(box[2] * height)), class_info[box[4]][1], 2)
 
        
    return img

In [ ]:
class_info = {}

class_info = {1: ['not-choice', (255,255,0)], 
              2: ['choice', (0,0,255)]}

In [ ]:
from PIL import Image
image = Image.fromarray(draw_bounding_boxes(image_np, output_dict,class_info), 'RGB')

8
[[0.81998897, 0.06863274, 0.8983147, 0.11531739, 1], [0.6460957, 0.06991642, 0.7184408, 0.11508442, 1], [0.46760225, 0.48646086, 0.53514314, 0.53071666, 1], [0.60241073, 0.4818774, 0.7174604, 0.53703713, 2], [0.46989894, 0.06889998, 0.53900415, 0.11594595, 1]]
3
[[0.46760225, 0.48646086, 0.53514314, 0.53071666, 1], [0.46989894, 0.06889998, 0.53900415, 0.11594595, 1], [0.60241073, 0.4818774, 0.7174604, 0.53703713, 2], [0.6460957, 0.06991642, 0.7184408, 0.11508442, 1], [0.81998897, 0.06863274, 0.8983147, 0.11531739, 1]]
0
1
2
2
[[0.46760225, 0.48646086, 0.53514314, 0.53071666, 1], [0.46989894, 0.06889998, 0.53900415, 0.11594595, 1], [0.6460957, 0.06991642, 0.7184408, 0.11508442, 1], [0.60241073, 0.4818774, 0.7174604, 0.53703713, 2]]
4 번을 선택했습니다.


In [ ]:
image.save("object_detection/data/test_choice.jpg")